# Building the POI dataset

The event siting tool uses points of interest (POI) data to indicate potential sites for their events. 

We use this notebook to demonstrate how we build an aggregate POI dataset from 4 sources:
- OpenStreetMap POI data for Florida
- OpenStreetMap Places of Worship (PofW) data for Florida
- IRS 990N filings for 2022 for small nonprofits
- IRS 990 filings for 2021 for large nonprofits

### Data sources

- Florida OSM data was downloaded manually via [geofabrik](http://download.geofabrik.de/north-america/us/florida.html) and placed in `data/external`
  - Warning: Raw unzipped data is 2.8GB
  - Detailed layer descriptions are available from geofabrik at https://www.geofabrik.de/data/geofabrik-osm-gis-standard-0.7.pdf
- IRS 990 and 990N data were downloaded from [irs.gov](https://www.irs.gov/charities-non-profits/form-990-series-downloads), parsed from XML format and geocoded separately, then placed in `data/interim`. 

The output file is saved to `data/interim/combined_pois.geojson`.

In [1]:
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

/Users/jtan/opt/anaconda3/envs/housd/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
# Set up data paths
DATA_DIR = "/Users/jtan/datakind/housd/data"

POI_PATH = os.path.join(DATA_DIR, "external", "florida-latest-free.shp", "gis_osm_pois_free_1.shp")

POFW_PATH = os.path.join(DATA_DIR, "external", "florida-latest-free.shp", "gis_osm_pofw_free_1.shp")

IRS990N_PATH = os.path.join(DATA_DIR, "interim", "fl_990n_addresses.csv")
IRS990N_GEO_PATH = os.path.join(DATA_DIR, "interim", "fl_990n_addresses_geocoded.csv")

IRS990_PATH = os.path.join(DATA_DIR, "interim", "IRS990_2021_mp_fl.csv")   
IRS990_GEO_PATH = os.path.join(DATA_DIR, "interim", "IRS990_2021_fl_geocoded.csv")   

## 1. Import data

In [3]:
pois = gpd.read_file(POI_PATH)

pois.head()

,osm_id,code,fclass,name,geometry
0,95855262,2422,camp_site,None,POINT (-85.90703 30.23211)
1,96919971,2724,memorial,None,POINT (-81.66089 30.32608)
2,97557631,2701,tourist_info,Mile 1,POINT (-85.41047 29.80190)
3,97571939,2007,library,Hardee Correctional Institution Library,POINT (-82.01485 27.59041)
4,98587924,2701,tourist_info,None,POINT (-82.96365 29.50284)


In [4]:
pofw = gpd.read_file(POFW_PATH)

pofw.head()

,osm_id,code,fclass,name,geometry
0,99977221,3102,christian_catholic,Blessed Sacrament Church,POINT (-82.79883 27.83294)
1,282705754,3100,christian,Crossing Church,POINT (-82.34436 27.92625)
2,283431303,3105,christian_methodist,Keeney Chapel United Methodist,POINT (-82.36945 27.93942)
3,286291955,3100,christian,Progress Village Seventh-day Adventist Church,POINT (-82.36733 27.89468)
4,286292175,3105,christian_methodist,Harris Temple United Methodist,POINT (-82.36657 27.89468)


In [5]:
irs_small = pd.read_csv(IRS990N_PATH)

irs_small.head()

,Unnamed: 0,EIN,Tax.Year,Organization.Name,"Gross.Receipts.Under.$25,000",Terminated,Tax.Period.Begin.Date,Tax.Period.End.Date,Website.URL,Officer.Name,...,Organization.Address.Line.1,Organization.Address.Line.2,Organization.Address.City,Organization.Address.Province,Organization.Address.State,Organization.Address.Postal.Code,Organization.Address.Country,Doing.Business.As.Name.1,Doing.Business.As.Name.2,Doing.Business.As.Name.3
0,2458,10600708,2021,INTERIOR COVERINGS MINISTRY INC,T,F,01-01-2021,12-31-2021,NaN,AUDREY DRUMMONDS,...,18235 ROSE STREET,NaN,GROVELAND,NaN,FL,34736,US,NaN,NaN,NaN
1,2654,10621975,2021,ANGELA COURTE MINISTRIES INC,T,F,01-01-2021,12-31-2021,NaN,Angela Mackenzie,...,272 West Cranes Circle,NaN,Altamonte Springs,NaN,FL,32701,US,NaN,NaN,NaN
2,3403,10706390,2021,NATIONAL FEDERATION OF MUSIC CLUBS,T,F,07-01-2021,06-30-2022,orlandomusicclub.org,Natalia Kalugina,...,303 East Highland Street,NaN,ALTAMONTE SPRINGS,NaN,FL,32701,US,NaN,NaN,NaN
3,3472,10716688,2021,ORLANDO KOREAN SENIOR CENTER INC,T,F,01-01-2021,12-31-2021,NaN,Alex Yi,...,506 ERICA WAY,NaN,WINTER SPRINGS,NaN,FL,32708,US,NaN,NaN,NaN
4,4422,10816505,2021,MISS RODEO FLORIDA ASSOCIATION INC,T,F,01-01-2021,12-31-2021,NaN,Frankie B Crawford,...,650 W Southport Rd,NaN,Kissimmee,NaN,FL,32746,US,NaN,NaN,NaN


In [6]:
irs_small_geo = pd.read_csv(IRS990N_GEO_PATH)

irs_small_geo.head()

,Unnamed: 0,id,geocoded_address,is_match,is_exact,returned_address,coordinates,tiger_line,side,state_fips,county_fips,tract,block,long,lat
0,0,263874780,"The Congaloosh Society, Kissimmee, FL, 34758",No_Match,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,202461223,"901 Webster Ave, Winter Park, FL, 32789",Match,Non_Exact,"901 W WEBSTER AVE, WINTER PARK, FL, 32789","-81.36138890599995,28.603997374000073",94276948.0,R,12.0,95.0,15901.0,2007.0,-81.361389,28.603997
2,2,260605478,"5749 NORMAN H CUTSON DRIVE, Orlando, FL, 32821",Match,Exact,"5749 NORMAN H CUTSON DR, ORLANDO, FL, 32821","-81.44954764099998,28.40610870300003",94335834.0,R,12.0,95.0,17026.0,1004.0,-81.449548,28.406109
3,3,203908198,"3080 Zaharias Drive, ORLANDO, FL, 32837",Match,Exact,"3080 ZAHARIAS DR, ORLANDO, FL, 32837","-81.41597000499996,28.36101755900006",617827390.0,R,12.0,95.0,17020.0,3000.0,-81.415970,28.361018
4,4,263483237,"909 Enclair, Orlando, FL, 32828",Match,Non_Exact,"909 ENCLAIR ST, ORLANDO, FL, 32828","-81.15631619499999,28.542850505000047",630991635.0,R,12.0,95.0,16748.0,1001.0,-81.156316,28.542851


In [7]:
irs_large = pd.read_csv(IRS990_PATH)

irs_large.head()

,org_ein,org_name1,org_name2,person_name,phone_num,addr,city,state,zip,officer_name,website,mission
0,650750873,THE MORTEZA-ZADEH FOUNDATION INC,NaN,NaN,NaN,700 NORTH OLIVE AVENUE,WEST PALM BEACH,FL,33401,NaN,NaN,NaN
1,204920710,PULICHINO TONG FAMILY FOUNDATION INC,NaN,NaN,5.614414e+09,2981 W MCNAB ROAD,POMPANO BEACH,FL,330694804,NaN,NaN,NaN
2,815129308,Breathe 150 Conference Inc,NaN,% Tracey Council,7.572771e+09,1315 Sand Lime Rd,Winter Garden,FL,34787,NaN,NaN,NaN
3,452641647,SUNCOAST AQUATIC NATURE CENTER,ASSOCIATES INC,NaN,9.413587e+09,5851 NATHAN BENDERSON CIR,SARASOTA,FL,34235,THOMAS HERRERA-MISHLER,WWW.NATHANBENDERSONPARK.ORG,THE ORGANIZATION'S PRIMARY PURPOSE IS TO MANAG...
4,593622890,PURE WORD MINISTRIES INC,NaN,% Richard J Perkins,NaN,2626 SUNNYSIDE CIR,PALM HARBOR,FL,346844159,NaN,NaN,NaN


In [8]:
irs_large_geo = pd.read_csv(IRS990_GEO_PATH)

irs_large_geo.head()

,id,geocoded_address,is_match,is_exact,returned_address,coordinates,tiger_line,side,state_fips,county_fips,tract,block,long,lat
0,526027312,"PO BOX 1908, ORLANDO, FL, 328021908",No_Match,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,223700343,"570 OCEAN DRIVE NO 1101, JUNO BEACH, FL, 33408",Match,Non_Exact,"570 OCEAN DR, JUNO BEACH, FL, 33408","-80.05262499199995,26.875237423000044",115507480.0,R,12.0,99.0,410.0,3001.0,-80.052625,26.875237
2,137529459,"471 NORTH ARROWHEAD TRAIL, VERO BEACH, FL, 32963",Match,Exact,"471 N ARROWHEAD TRL, VERO BEACH, FL, 32963","-80.38898884799994,27.740958351000074",120326359.0,L,12.0,61.0,50506.0,2002.0,-80.388989,27.740958
3,465314273,"200 W COLLEGE AVENUE NO 210, TALLAHASSEE, FL, ...",No_Match,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,596178235,"803 EYRIE DRIVE 101, OVIEDO, FL, 32765",Match,Exact,"803 EYRIE DR, OVIEDO, FL, 32765","-81.22313752499997,28.658404939000036",93746631.0,L,12.0,117.0,21314.0,3022.0,-81.223138,28.658405


## 2. Transform datasets to fit a common schema

The final POI dataset will have the following schema:
- id
- name: name of the location
- type: the type of location this is
- geometry: a shapely Point object
- source: where the data came from

### 2.1 POI data

In [9]:
pois.columns

Index(['osm_id', 'code', 'fclass', 'name', 'geometry'], dtype='object')

In [10]:
# Create crosswalk between old and new column names
cols_to_keep = {
    "osm_id": "id",
    "name": "name",
    "fclass": "type",
    "geometry": "geometry"
}

pois = pois[cols_to_keep.keys()].rename(columns=cols_to_keep)
pois["source"] = "openstreetmap"

pois.head()

,id,name,type,geometry,source
0,95855262,None,camp_site,POINT (-85.90703 30.23211),openstreetmap
1,96919971,None,memorial,POINT (-81.66089 30.32608),openstreetmap
2,97557631,Mile 1,tourist_info,POINT (-85.41047 29.80190),openstreetmap
3,97571939,Hardee Correctional Institution Library,library,POINT (-82.01485 27.59041),openstreetmap
4,98587924,None,tourist_info,POINT (-82.96365 29.50284),openstreetmap


### 2.2 PofW data

In [11]:
pofw.columns

Index(['osm_id', 'code', 'fclass', 'name', 'geometry'], dtype='object')

In [12]:
# Create crosswalk between old and new column names
cols_to_keep = {
    "osm_id": "id",
    "name": "name",
    "fclass": "type",
    "geometry": "geometry"
}

pofw = pofw[cols_to_keep.keys()].rename(columns=cols_to_keep)

pofw["type"] = "place_of_worship"
pofw["source"] = "openstreetmap"

pofw.head()

,id,name,type,geometry,source
0,99977221,Blessed Sacrament Church,place_of_worship,POINT (-82.79883 27.83294),openstreetmap
1,282705754,Crossing Church,place_of_worship,POINT (-82.34436 27.92625),openstreetmap
2,283431303,Keeney Chapel United Methodist,place_of_worship,POINT (-82.36945 27.93942),openstreetmap
3,286291955,Progress Village Seventh-day Adventist Church,place_of_worship,POINT (-82.36733 27.89468),openstreetmap
4,286292175,Harris Temple United Methodist,place_of_worship,POINT (-82.36657 27.89468),openstreetmap


### 2.3 IRS 990N data (`irs_small`)

In [13]:
# Create geodataframe of shapely Points from lat-long 
irs_small_gdf = gpd.GeoDataFrame(
    irs_small, 
    geometry=gpd.points_from_xy(irs_small_geo.long, irs_small_geo.lat)
).set_crs(4326)

irs_small_gdf.head()

,Unnamed: 0,EIN,Tax.Year,Organization.Name,"Gross.Receipts.Under.$25,000",Terminated,Tax.Period.Begin.Date,Tax.Period.End.Date,Website.URL,Officer.Name,...,Organization.Address.Line.2,Organization.Address.City,Organization.Address.Province,Organization.Address.State,Organization.Address.Postal.Code,Organization.Address.Country,Doing.Business.As.Name.1,Doing.Business.As.Name.2,Doing.Business.As.Name.3,geometry
0,2458,10600708,2021,INTERIOR COVERINGS MINISTRY INC,T,F,01-01-2021,12-31-2021,NaN,AUDREY DRUMMONDS,...,NaN,GROVELAND,NaN,FL,34736,US,NaN,NaN,NaN,POINT EMPTY
1,2654,10621975,2021,ANGELA COURTE MINISTRIES INC,T,F,01-01-2021,12-31-2021,NaN,Angela Mackenzie,...,NaN,Altamonte Springs,NaN,FL,32701,US,NaN,NaN,NaN,POINT (-81.36139 28.60400)
2,3403,10706390,2021,NATIONAL FEDERATION OF MUSIC CLUBS,T,F,07-01-2021,06-30-2022,orlandomusicclub.org,Natalia Kalugina,...,NaN,ALTAMONTE SPRINGS,NaN,FL,32701,US,NaN,NaN,NaN,POINT (-81.44955 28.40611)
3,3472,10716688,2021,ORLANDO KOREAN SENIOR CENTER INC,T,F,01-01-2021,12-31-2021,NaN,Alex Yi,...,NaN,WINTER SPRINGS,NaN,FL,32708,US,NaN,NaN,NaN,POINT (-81.41597 28.36102)
4,4422,10816505,2021,MISS RODEO FLORIDA ASSOCIATION INC,T,F,01-01-2021,12-31-2021,NaN,Frankie B Crawford,...,NaN,Kissimmee,NaN,FL,32746,US,NaN,NaN,NaN,POINT (-81.15632 28.54285)


In [14]:
# Create crosswalk between old and new column names
cols_to_keep = {
    "EIN": "id", 
    "Organization.Name": "name",
    "geometry": "geometry"
}

# Drop NAs 
irs_small_gdf = irs_small_gdf[~irs_small_geo.lat.isna()]

# Apply crosswalk
irs_small_gdf = irs_small_gdf[cols_to_keep.keys()].rename(columns=cols_to_keep)

# Create "type" and "source" column
irs_small_gdf["type"] = "nonprofit"
irs_small_gdf["source"] = "IRS 990N"

irs_small_gdf.head()

,id,name,geometry,type,source
1,10621975,ANGELA COURTE MINISTRIES INC,POINT (-81.36139 28.60400),nonprofit,IRS 990N
2,10706390,NATIONAL FEDERATION OF MUSIC CLUBS,POINT (-81.44955 28.40611),nonprofit,IRS 990N
3,10716688,ORLANDO KOREAN SENIOR CENTER INC,POINT (-81.41597 28.36102),nonprofit,IRS 990N
4,10816505,MISS RODEO FLORIDA ASSOCIATION INC,POINT (-81.15632 28.54285),nonprofit,IRS 990N
6,10877308,SHRINERS INTERNATIONAL,POINT (-80.80127 28.55893),nonprofit,IRS 990N


### 2.4 IRS 990 data (`irs_large`)

In [15]:
# Drop NAs
irs_large_geo = irs_large_geo[~irs_large_geo.lat.isna()]

# Create geodataframe of shapely Points from lat-long 
irs_large_gdf = gpd.GeoDataFrame(
    irs_large_geo, 
    geometry=gpd.points_from_xy(irs_large_geo.long, irs_large_geo.lat)
).set_crs(4326)

irs_large_gdf.head()

,id,geocoded_address,is_match,is_exact,returned_address,coordinates,tiger_line,side,state_fips,county_fips,tract,block,long,lat,geometry
1,223700343,"570 OCEAN DRIVE NO 1101, JUNO BEACH, FL, 33408",Match,Non_Exact,"570 OCEAN DR, JUNO BEACH, FL, 33408","-80.05262499199995,26.875237423000044",115507480.0,R,12.0,99.0,410.0,3001.0,-80.052625,26.875237,POINT (-80.05262 26.87524)
2,137529459,"471 NORTH ARROWHEAD TRAIL, VERO BEACH, FL, 32963",Match,Exact,"471 N ARROWHEAD TRL, VERO BEACH, FL, 32963","-80.38898884799994,27.740958351000074",120326359.0,L,12.0,61.0,50506.0,2002.0,-80.388989,27.740958,POINT (-80.38899 27.74096)
4,596178235,"803 EYRIE DRIVE 101, OVIEDO, FL, 32765",Match,Exact,"803 EYRIE DR, OVIEDO, FL, 32765","-81.22313752499997,28.658404939000036",93746631.0,L,12.0,117.0,21314.0,3022.0,-81.223138,28.658405,POINT (-81.22314 28.65840)
7,592174510,"6401 Lyons Road, Coconut Creek, FL, 330733602",Match,Exact,"6401 LYONS RD, COCONUT CREEK, FL, 33073","-80.18657224699996,26.298654316000068",114798550.0,R,12.0,11.0,10606.0,1000.0,-80.186572,26.298654,POINT (-80.18657 26.29865)
9,592244943,"3160 Southgate Commerce Blvd Suite, Orlando, F...",Match,Exact,"3160 SOUTHGATE COMMERCE BLVD, ORLANDO, FL, 32806","-81.37714689099994,28.50756628700003",641064641.0,L,12.0,95.0,13900.0,1019.0,-81.377147,28.507566,POINT (-81.37715 28.50757)


In [16]:
# Create crosswalk between old and new column names
cols_to_keep = {
    "org_ein": "id",
    "org_name1": "name",
    "geometry": "geometry"
}

# Merge on geo data, filter and rename columns
irs_large_gdf = irs_large.merge(irs_large_gdf, how="inner", left_on="org_ein", right_on="id") \
    [cols_to_keep.keys()] \
    .rename(columns=cols_to_keep)

# Create "type" and "source" column
irs_large_gdf["type"] = "nonprofit"
irs_large_gdf["source"] = "IRS 990"

irs_large_gdf.head()

,id,name,geometry,type,source
0,650750873,THE MORTEZA-ZADEH FOUNDATION INC,POINT (-80.05177 26.71895),nonprofit,IRS 990
1,204920710,PULICHINO TONG FAMILY FOUNDATION INC,POINT (-80.18108 26.20917),nonprofit,IRS 990
2,815129308,Breathe 150 Conference Inc,POINT (-81.59647 28.54708),nonprofit,IRS 990
3,593622890,PURE WORD MINISTRIES INC,POINT (-82.72892 28.05746),nonprofit,IRS 990
4,387193282,MARK & EILEEN EPSTEIN FAMILY CHARITABLE,POINT (-80.27637 25.32272),nonprofit,IRS 990


## 3. Combine all datasets

In [17]:
datasets = [pois, pofw, irs_small_gdf, irs_large_gdf]

all_pois = pd.concat(datasets)

all_pois.head()

,id,name,type,geometry,source
0,95855262,None,camp_site,POINT (-85.90703 30.23211),openstreetmap
1,96919971,None,memorial,POINT (-81.66089 30.32608),openstreetmap
2,97557631,Mile 1,tourist_info,POINT (-85.41047 29.80190),openstreetmap
3,97571939,Hardee Correctional Institution Library,library,POINT (-82.01485 27.59041),openstreetmap
4,98587924,None,tourist_info,POINT (-82.96365 29.50284),openstreetmap


## 4. Save data

In [18]:
OUTPUT_PATH = os.path.join(DATA_DIR, "interim", "combined_pois.geojson")

all_pois.to_file(OUTPUT_PATH, driver="GeoJSON")

/Users/jtan/opt/anaconda3/envs/housd/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
